In [ ]:
import pandas as pd

df = pd.read_csv("Laptop Dataset.csv", quotechar='"', on_bad_lines='skip', engine='python')
df.head()

,rating,review,Unnamed: 2,Unnamed: 3
0,5,"Loved it, it's my first MacBook that I earned ...",NaN,NaN
1,5,Battery lasted longer than my first relationsh...,NaN,NaN
2,5,Such a great deal.. very happy with the perfor...,NaN,NaN
3,4,"Awesome build quality and very good display, b...",NaN,NaN
4,5,When i ordered and came to know about seller r...,NaN,NaN


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, temp_texts , train_labels, temp_labels  = train_test_split(
    df['review'].values,
    df['rating'].apply(lambda x: 1 if x >= 4 else 0).values,
    test_size=0.2,
    random_state=42
)

In [ ]:
import numpy as np
import tensorflow as tf

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts,
    temp_labels,
    test_size=0.5,
    random_state=42
)
train_data = tf.data.Dataset.from_tensor_slices((train_texts, train_labels)).shuffle(10000).batch(512)
val_data = tf.data.Dataset.from_tensor_slices((val_texts, val_labels)).batch(512)
test_data = tf.data.Dataset.from_tensor_slices((test_texts, test_labels)).batch(512)

In [ ]:
import tensorflow_hub as hub

embedding="https://tfhub.dev/google/universal-sentence-encoder/4"
hub_layer=hub.KerasLayer(embedding,input_shape=[],dtype=tf.string,trainable=True)

# Define a custom Keras Model that includes the hub_layer
class TextClassificationModel(tf.keras.Model):
    def __init__(self, hub_layer, num_hidden_units=16, num_output_units=1):
        super(TextClassificationModel, self).__init__()
        self.hub_layer = hub_layer
        self.dense1 = tf.keras.layers.Dense(num_hidden_units, activation='relu')
        self.dense2 = tf.keras.layers.Dense(num_output_units, activation='sigmoid')

    def call(self, inputs):
        embeddings = self.hub_layer(inputs)
        x = self.dense1(embeddings)
        return self.dense2(x)

model = TextClassificationModel(hub_layer)

model.summary()

Model: "text_classification_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,
                    epochs=20,
                    validation_data=val_data,
                    verbose=1)

Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - accuracy: 0.8025 - loss: 0.6048 - val_accuracy: 0.8146 - val_loss: 0.4297
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 132ms/step - accuracy: 0.8055 - loss: 0.4198 - val_accuracy: 0.8291 - val_loss: 0.3601
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 100ms/step - accuracy: 0.8422 - loss: 0.3539 - val_accuracy: 0.8785 - val_loss: 0.3113
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.8826 - loss: 0.3153 - val_accuracy: 0.8951 - val_loss: 0.2782
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 130ms/step - accuracy: 0.8933 - loss: 0.2884 - val_accuracy: 0.9000 - val_loss: 0.2573
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - accuracy: 0.9028 - loss: 0.2692 - val_accuracy: 0.9088 - val_loss: 0.2448
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - accuracy: 0.9110 - loss: 0.2523 - val_accuracy: 0.9129 - val_loss: 0.2364
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 118ms/step - accuracy: 0.9081 - loss: 0.2560 - val_accuracy: 0.91

In [ ]:
result=model.evaluate(test_data,verbose=2)
for name,value in zip(model.metrics_names,result):
  print("%s:%.3f"%(name,value))

5/5 - 0s - 81ms/step - accuracy: 0.9270 - loss: 0.2005
loss:0.201
compile_metrics:0.927


In [ ]:
print("Enter the Movie Review to check if it is a good review or not: ")
search=input("")
search_batch=tf.convert_to_tensor([search])

Enter the Movie Review to check if it is a good review or not: 
This laptop was is the best


In [ ]:
prediction=model.predict(search_batch)
if prediction[0][0] >= 0.5:
    print("This looks like a POSITIVE review. 😄")
else:
    print("This looks like a NEGATIVE review. 😢")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 946ms/step
This looks like a POSITIVE review. 😄
